# Langmuir isotherm predictions improved by FoKL-GPy in benchmark comparison of temperature-dependent model

---

## Abstract

Expanding *Langmuir isotherm predictions improved by FoKL-GPy in benchmark comparison* to include temperature, the site fraction at any temperature and pressure pair may be predicted.

---

## Contents

- 1
- 2
- 3

---

## Method

From *Langmuir isotherm predictions improved by FoKL-GPy in benchmark comparison*, the following is known and will provide the Gaussian process (GP) model $\mathbf{GP}$ trained on $\Delta$ as a function of the inverse of temperature $T$ and the natural logarithim of pressure $p$.

$\begin{equation}\tag{1} \mathbf{GP} = \{\Delta\ |\ T^{-1},\ \ln(p)\} \end{equation}$

$\begin{equation}\tag{2} \Delta = \ln(\sigma_1) \end{equation}$

$\begin{equation}\tag{3} \sigma_1 = \frac{q}{\mu(q_{max}-q)} \end{equation}$

$\begin{equation}\tag{4} \mu = \frac{p}{\sqrt{2 \pi M R T}} \end{equation}$

$\begin{equation}\tag{5} q_{max} = b \end{equation}$

The following is also known and will provide predictions of $q$, termed $q_{cal}$, from the GP.

$\begin{equation}\tag{6} q_{cal} = \frac{\sigma_{1, cal} q_{max} \mu}{1 + \sigma_{1, cal} \mu} \end{equation}$

$\begin{equation}\tag{7} \sigma_{1, cal} = \exp(\mathbf{GP}) \end{equation}$

The values of $q_{cal}$ found with the GP may then be compared to $q_{cal}$ of Langmuir's mica experiment; specifically, for $N_2$. The experimental measurements of $q(p)$ for $N_2$ may be found in Table II ($90 \text{ K}$) and Table X ($155 \text{ K}$), though this data has been copied to *N2.txt*.

---

## Implementation:

Importing modules and defining constants:

In [1]:
import os
import sys
sys.path.append(os.path.join('..', '..', '..'))  # include local path (i.e., this package)
from src.FoKL import FoKLRoutines
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate


SI_scale = True
logscale = True
save = False

M = 0.0280134  # molecular weight of N2, kg / mol
if SI_scale is True:
    R = 8.31446261815324  # universal gas constant, J / (K * mol)
else:
    R = 8314462.61815324  # universal gas constant, mN mm / (K * mol)

/home/jacobpatrick/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Loading and parsing the data:

In [2]:
data = np.loadtxt(os.path.join('data', 'N2.txt'), skiprows=1)

T = data[:, 0]
p = data[:, 1]
q = data[:, 2]
a_linear = data[:, 3]  # for plotting linear fit
qmax = data[:, 4]  # == b
qcal_linear = data[:, 5]  # for benchmark comparison

Defining equations:

In [3]:
def eq2(sigma1):
    """Delta"""
    return np.log(sigma1)

def eq3(q, mu, qmax):
    """sigma1"""
    return q / mu / (qmax - q)

def eq4(p, M, R, T):
    """mu"""
    return p / np.sqrt(2 * np.pi * M * R * T)

Calculating the training data $\{\Delta\ |\ T^{-1},\ \ln(p)\}$:

In [4]:
mu = eq4(p, M, R, T)
sigma1 = eq3(q, mu, qmax)
Delta = eq2(sigma1)

inv_T = 1 / T
ln_p = np.log(p)

Training the GP model:

In [6]:
model = FoKLRoutines.FoKL(kernel=1)
_ = model.fit([inv_T, ln_p], Delta, tolerance=12, clean=True)

/home/jacobpatrick/FoKL-GPy/docs/_dev/langmuir_isotherm/../../../src/FoKL/FoKLRoutines.py:299: UserWarning: 'inputs' was transposed. Ignore if more datapoints than input variables, else set 'AutoTranspose=False' to disable.
  warnings.warn("'inputs' was transposed. Ignore if more datapoints than input variables, else set "


[1, -15.130113256172816]
[2, -15.130113256172816]
[2, -15.130113256172816]
[3, -15.130113256172816]
[3, -15.130113256172816]
[4, -15.130113256172816]
[4, -15.130113256172816]
[4, -15.531740084434627]
[5, -15.531740084434627]
[5, -15.531740084434627]
[5, -15.531740084434627]
[6, -15.531740084434627]
[6, -15.531740084434627]
[6, -15.632623916360416]
[6, -15.632623916360416]
[7, -24.686210667603234]
[7, -409.5710379492436]
[7, -409.5710379492436]
[7, -134.2176436974426]
[8, -134.2176436974426]
[8, -134.2176436974426]
[8, -134.2176436974426]
[8, -134.2176436974426]
[8, -134.2176436974426]
[9, -135.83340390407747]
[9, -135.83340390407747]
[9, -135.83340390407747]
[9, -96.86621831745401]
[9, -96.86621831745401]


Stuff:

NEW DATASET TO USE SO THAT MULTIPLE TEMPS ARE AVAILABLE:

-  https://link.springer.com/article/10.1007/s11356-023-31820-x
    - fig 1
        - points to: https://www.sciencedirect.com/science/article/pii/S138589471000731X
            - couldn't find dataset at first glance